In [1]:
# add our custom modules to the python import path
import sys
import os
import numpy as np
from qick import *
sys.path.append(os.getcwd() + '/../../Client_modules/')

import mResSweep
from socProxy import makeProxy
import h5py
from PythonDrivers.control_atten import setatten
%matplotlib inline
import matplotlib.pyplot as plt

# Python 3.7
# os.environ['PATH'] = os.getcwd() + '\\PythonDrivers' + os.pathsep + os.environ['PATH']
# Python 3.8 and newer
os.add_dll_directory(os.getcwd() + '/../../Client_modules/PythonDrivers')

Could not import QickSoc: No module named 'pynq'


<AddedDllDirectory('C:\\Users\\my\\Documents\\GitHub\\ZCU216_Russell\\res_dev\\notebooks\\2022-10-16 ZCU216 loopback/../../Client_modules/PythonDrivers')>

In [2]:
# create proxy to ZCU216 board
soc, soccfg = makeProxy('192.168.1.110')
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@192.168.1.110:8888
myqick PYRO:obj_8c825da9ba87403da4ed24762a70f40e@192.168.1.110:33513

QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v4 - tProc output 1, switch ch 0, maxlen 65536
		DAC tile 2, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	1:	axis_signal_gen_v4 - tProc output 2, switch ch 1, maxlen 65536
		DAC tile 2, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	2:	axis_signal_gen_v4 - tProc output 3, switch ch 2, maxlen 65536
		DAC tile 2, ch 2, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	3:	axis_signal_gen_v4 - tProc output 4, switch ch 3, maxlen 65536
		DAC tile 2, ch 3, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	4:	axis_signal_gen_v4 - tProc output 5, switch ch 4, maxlen 65536
		DAC tile 3, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	5:	axis_signal_gen_v4 - tProc output 6, switch ch 5, maxlen 65536
	

# define the program

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]
        
        freqList = [self.reg2freq(self.freq2reg(freq,gen_ch=res_ch, ro_ch=cfg["ro_chs"][i]),gen_ch=res_ch) for i, freq in enumerate(cfg['pulse_freqs'])]

        
        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1, mixer_freq=cfg["mixer_freq"], mux_freqs=cfg["pulse_freqs"],
                          mux_gains=cfg["pulse_gains"], ro_ch=cfg["ro_chs"][0])
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for iCh, ch in enumerate(cfg["ro_chs"]):
            self.declare_readout(ch=ch, freq=cfg["pulse_freqs"][iCh], length=cfg["readout_length"],
                                 gen_ch=cfg["res_ch"])

        self.set_pulse_registers(ch=cfg["res_ch"], style="const", length=cfg["length"], 
                                 mask=[0, 1, 2, 3])
        
        self.sync_all(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.cfg["relax_delay"])
        
        # equivalent to the following:
#         self.trigger(adcs=self.ro_chs,
#                      pins=[0], 
#                      adc_trig_offset=self.cfg["adc_trig_offset"])
#         self.pulse(ch=self.cfg["res_ch"])
#         self.wait_all()
#         self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

# run a single point, decimated

In [ ]:
config={"res_ch":6, # --Fixed
        "ro_chs":[0,1], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":500, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":2000, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":1000, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gains": [1,1], # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units
        
        "mixer_freq": 2000,

        "pulse_freqs": [0, 500], # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 75,# [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":10
        # Try varying soft_avgs from 1 to 200 averages

       }
# for i, f in enumerate(config['pulse_freqs']):
#             config['pulse_freqs'][i] = soc.roundfreq(f, dict1=soccfg['gens'][6],
#                                                 dict2=soccfg['readouts'][config['ro_chs'][i]])        

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

In [ ]:
# Plot results.
for ii, iq in enumerate(iq_list):
    plt.figure(ii)
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
    plt.ylabel("a.u.")
    plt.xlabel("Clock ticks")
    plt.title("Averages = " + str(config["soft_avgs"]))
    plt.legend()

# run single point

In [ ]:
config={"res_ch":6, # --Fixed
        "ro_chs":[0,1], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":500, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":2000, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":1000, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gains": [1,1], # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units
        
        "mixer_freq": 2000,

        "pulse_freqs": [0, 500], # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 75,# [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks
       }
# for i, f in enumerate(config['pulse_freqs']):
#             config['pulse_freqs'][i] = soc.roundfreq(f, dict1=soccfg['gens'][6],
#                                                 dict2=soccfg['readouts'][config['ro_chs'][i]])        

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

for ii, iq in enumerate(iq_list):
    print(iq[0][0])
    print('Freq {0}, I = {1:0.2f}'.format(ii, iq[0][0]))
    print('Freq {0}, Q = {1:0.2f}'.format(ii, iq[1][0]))
    print('Freq {0}, phase = {1:0.2f}'.format(ii, np.arctan2(iq[1][0],iq[0][0])))
    print('')


# run two points

In [ ]:
config={"res_ch":6, # --Fixed
        "ro_chs":[0,1], # --Fixed
        "reps":10, # --Fixed
        "relax_delay":5000, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":2000, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":1000, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gains": [1,1], # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units
        
        "mixer_freq": 2000,

        "pulse_freqs": [0, 500], # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 200,# [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks
       }

prog =LoopbackProgram(soccfg, config)
iq_list1 = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

config['pulse_freqs'] = [-100, 400]

prog =LoopbackProgram(soccfg, config)
iq_list2 = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

config['pulse_freqs'] = [0, 500]

prog =LoopbackProgram(soccfg, config)
iq_list3 = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

for ii, iq in enumerate(iq_list1):
#     print('Freq {0}, I = {1:0.2f}'.format(ii, iq[0][0]))
#     print('Freq {0}, Q = {1:0.2f}'.format(ii, iq[1][0]))
    print('Freq {0}, phase = {1:0.2f}'.format(ii, np.arctan2(iq[1][0],iq[0][0])))
    print('Freq {0}, mag = {1:0.2f}'.format(ii, np.abs(iq[1][0]+1j*iq[0][0])))
    print('')
    
for ii, iq in enumerate(iq_list2):
#     print('Freq {0}, I = {1:0.2f}'.format(ii, iq[0][0]))
#     print('Freq {0}, Q = {1:0.2f}'.format(ii, iq[1][0]))
    print('Freq {0}, phase = {1:0.2f}'.format(ii, np.arctan2(iq[1][0],iq[0][0])))
    print('Freq {0}, mag = {1:0.2f}'.format(ii, np.abs(iq[1][0]+1j*iq[0][0])))
    print('')
    
for ii, iq in enumerate(iq_list3):
#     print('Freq {0}, I = {1:0.2f}'.format(ii, iq[0][0]))
#     print('Freq {0}, Q = {1:0.2f}'.format(ii, iq[1][0]))
    print('Freq {0}, phase = {1:0.2f}'.format(ii, np.arctan2(iq[1][0],iq[0][0])))
    print('Freq {0}, mag = {1:0.2f}'.format(ii, np.abs(iq[1][0]+1j*iq[0][0])))
    print('')
